In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
#import random
from scipy.signal import convolve2d
from scipy.ndimage.filters import laplace

### Função para display de lista de imagens

In [2]:
def disp(f,dpi=72):
    images=[]
    for f_i in f:
        images.append(Image.fromarray(np.uint8(f_i)))
 
    # tamanho do display de cada imagem 
    _dpi=dpi  # padrao da funcao display da biblioteca PIL
    altura,largura=images[0].size # em pixels (assumimos o mesmo tamanho para todas)
    figsizeinches=largura/float(_dpi),altura/float(_dpi) # em polegadas
    fig = plt.figure(figsize=figsizeinches)
    
    # valores iniciais
    xpos=0
    ypos=0
    espacamento=0.02
    tam_y=largura/largura
    tam_x=altura/largura

    #mod
    max_row_size=4
    i=0
    
    for image in images:
        ax=fig.add_axes([xpos,ypos,tam_x,tam_y]) 
        ax.axis('off')
        ax.imshow(image)
        
        # incrementos
        i=i+1                  # contagem de figuras
        if i%max_row_size==0:  # troca de linha
            xpos=0
            ypos=ypos-tam_y-espacamento
        else:                  # continua na linha
            xpos=xpos+tam_x+espacamento
    plt.show()

## Método 1. Inpainting utilizando a equação de calor

In [3]:
def _heat_inpaint(f,antimask,n=None,epsilon=0.001):
    # antimask: 0 dentro de omega
    #         : 1 fora de omega
    # mask    : 1 dentro de omega
    #         : 0 fora de omega
    
    # tratamento da máscara
    T=np.zeros(256)
    T[0]=1
    mask=T[antimask.astype('uint8')].astype('uint8')
    
    f=f*(mask^1)
    
    alpha=0.1

    restauracao=f.copy()
    if n!=None:
        for i in range(0,n):
            f_t=alpha*laplace(restauracao)
            restauracao=restauracao+f_t*mask
    else:
        i=0
        while True:
            f_t=alpha*laplace(restauracao)
            restauracao_atual=restauracao+f_t*mask
            i=i+1
            err_mat=np.abs(restauracao_atual-restauracao)
            err=sum(map(sum,err_mat))
            if err<epsilon:
                break
            restauracao=restauracao_atual
    return restauracao.astype('uint8')

## Método 2. Inpainting utilizando filtro gaussiano (fast inpainting)

In [4]:
def _fast_inpaint(f,antimask,n=None,last=True,epsilon=0.001):
    # antimask: 0 dentro de omega
    #         : 1 fora de omega
    # mask: 1 dentro de omega
    #       0 fora de omega
    
    # tratamento da máscara
    T=np.zeros(256)
    T[0]=1
    mask=T[antimask.astype('uint8')].astype('uint8')
    f=f*(mask^1)

    #a=0.073235
    #b=0.176765
    #kernel=np.array([[a,b,a],[b,0,b],[a,b,a]])

    a=1/16
    b=1/8
    c=1/4
    kernel=np.array([[a,b,a],[b,c,b],[a,b,a]])
    
    restauracao=f.copy()
    if n!=None:
        for i in range(0,n):
            f_t=convolve2d(restauracao,kernel,mode='same')
            restauracao=f+f_t*mask
    else:
        i=0
        while True:
            f_t=convolve2d(restauracao,kernel,mode='same')
            restauracao_atual=f+f_t*mask
            i=i+1
            err_mat=np.abs(restauracao_atual-restauracao)
            err=sum(map(sum,err_mat))
            if err<epsilon:
                break
            restauracao=restauracao_atual
        
    return restauracao.astype('uint8')

## Inpainting para imagens coloridas

In [5]:
def inpaint_rgb(f_rgb,mask_rgb,mode='fast',n=None,e=0.01):
    # canais da imagem
    f_r=f_rgb[:,:,0]
    f_g=f_rgb[:,:,1]
    f_b=f_rgb[:,:,2]
    
    # canais da máscara
    mask_r=mask_rgb[:,:,0]
    mask_g=mask_rgb[:,:,1]
    mask_b=mask_rgb[:,:,2]
    
    # modes: 
    #   'heat' : usa _heat_inpaint()
    #   'fast' : usa _fast_inpaint()
    if mode=='heat':
        restauracao_r=_heat_inpaint(f_r,mask_r,n=n,epsilon=e)
        restauracao_g=_heat_inpaint(f_g,mask_g,n=n,epsilon=e)
        restauracao_b=_heat_inpaint(f_b,mask_b,n=n,epsilon=e)
    elif mode=='fast':
        restauracao_r=_fast_inpaint(f_r,mask_r,n=n,epsilon=e)
        restauracao_g=_fast_inpaint(f_g,mask_g,n=n,epsilon=e)
        restauracao_b=_fast_inpaint(f_b,mask_b,n=n,epsilon=e)
        
    restauracao_rgb=np.zeros(f_rgb.shape)
    restauracao_rgb[:,:,0]=restauracao_r
    restauracao_rgb[:,:,1]=restauracao_g
    restauracao_rgb[:,:,2]=restauracao_b
    
    return restauracao_rgb.astype('uint8')